# Introducción 
Los sistemas de recomendación, a veces llamados en inglés “recommender systems” son algoritmos que intentan “predecir” los siguientes ítems (productos, canciones, etc.) que querrá adquirir un usuario en particular.
 
Entre las estrategias más usadas para crear sistemas de recomendación encontramos:
* **Popularity**: Aconseja por la “popularidad” de los productos. Por ejemplo, “los más vendidos” globalmente, se ofrecerán a todos los usuarios por igual sin aprovechar la personalización. Es fácil de implementar y en algunos casos es efectiva.
* **Content-based**: A partir de productos visitados por el usuario, se intenta “adivinar” qué busca el usuario y ofrecer mercancías similares.
* **Colaborative**: Es el más novedoso, pues utiliza la información de “masas” para identificar perfiles similares y aprender de los datos para recomendar productos de manera individual.
 
En esta ocasión usaremos la metodologia de Collaborative Filtering.
El Collaborative Filtering se basa en crear una matriz en la que pondremos a los usuarios en las filas y los items en columnas de modo que cada usuario pondrá una calificación a cada producto, evidentemente tendremos huecos en la matriz pues evidentemente no todos los usuarios tienen o valoraron todos los ítems.
Una matriz con muchas celdas vacías se dice -en inglés- que es sparce (y suele ser normal) en cambio si tuviéramos la mayoría de las celdas cubiertas con valoraciones, se llamará dense.
## Tipos de Collaborative Filtering
* **User-based**:
    * Se identifican usuarios similares
    * Se recomiendan nuevos ítems a otros usuarios basado en el rating dado por otros usuarios similares (que no haya valorado este usuario)
* **Item-based**: (Este es el que veremos a continuación)
    * Calcular la similitud entre items
    * Encontrar los “mejores items similares” a los que un usuario no tenga evaluados y recomendárselos.

Veamos ahora la aplicación del algoritmo en python:

Antes de empezar cabe resaltar que necesitamos los siguientes archivos para empezar,los puedes encontrar aqui: https://drive.google.com/drive/folders/10P7C-suaBAA5_ZAi4oEO_Pmsb7kG6JMZ 

Veamos el código:


In [1]:
import numpy as np # Librerias a importar 
import pandas as pd # Librerias a importar 

# Importamos nuestros archivos 
creditos_df = pd.read_csv('credits.csv') 
peliculas_df = pd.read_csv('movies.csv') 
# Comprobamos archivos y que tengan los mismos titulos 
print (creditos_df.head()) # Revisión de información 
print (peliculas_df.head()) # Revisión de información 


   movie_id  ...                                               crew 
0     19995  ...  [{'credit_id': '52fe48009251416c750aca23', 'de... 
1       285  ...  [{'credit_id': '52fe4232c3a36847f800b579', 'de... 
2    206647  ...  [{'credit_id': '54805967c3a36829b5002c41', 'de... 
3     49026  ...  [{'credit_id': '52fe4781c3a36847f81398c3', 'de... 
4     49529  ...  [{'credit_id': '52fe479ac3a36847f813eaa3', 'de... 

[5 rows x 4 columns] 
      budget  ... vote_count 
0  237000000  ...      11800 
1  300000000  ...       4500 
2  245000000  ...       4466 
3  250000000  ...       9106 
4  260000000  ...       2124 

[5 rows x 20 columns] 


In [2]:
# Hacemos una pausa para verificar que ambos archivos tengan los mismos titulos, como ambos difieren en tamaño de titulos los juntamos en uno solo 
peliculas_df = peliculas_df.merge(creditos_df, on = 'title') #Agregamos tres columnas, 'movie_id', 'cast' y 'crew' comparandola con la casilla 'title', por eso se usó 'merge' 
print (peliculas_df.shape) # Revisión de información 
print (peliculas_df.head()) # Revisión de información 
print (peliculas_df.info()) # Revisión de información 


(4808, 23) 
      budget  ...                                               crew 
0     19995  ...  [{'credit_id': '52fe48009251416c750aca23', 'de... 
1       285  ...  [{'credit_id': '52fe4232c3a36847f800b579', 'de... 
2    206647  ...  [{'credit_id': '54805967c3a36829b5002c41', 'de... 
3     49026  ...  [{'credit_id': '52fe4781c3a36847f81398c3', 'de... 
4     49529  ...  [{'credit_id': '52fe479ac3a36847f813eaa3', 'de... 
 
[5 rows x 23 columns] 
<class 'pandas.core.frame.DataFrame'> 
Int64Index: 4808 entries, 0 to 4807 
Data columns (total 23 columns): 
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   budget                4808 non-null   int64   
 1   genres                4808 non-null   object  
 2   homepage              1713 non-null   object  
 3   id                    4808 non-null   int64   
 4   keywords              4808 non-null   object  
 5   original_language     4808 non-null   object  
 6   original_title   

In [3]:
# Una vez confirmamos que tenemos los mismos titulos en un mismo archivo nos quedamos solo con unas columnas 
peliculas_df = peliculas_df[['movie_id','title','overview','genres','keywords','cast','crew']] 
print (peliculas_df.head()) # Revisión de información 
print (peliculas_df.info()) # Revisión de información 
print (peliculas_df.isnull().sum()) # Nos muestra los 'Null' en cada columna 
peliculas_df.dropna(inplace = True) 
print (peliculas_df.duplicated().sum()) # Nos dice las filas duplicadas 
print (peliculas_df.iloc[0].genres) # Nos devuelve el contenido de la primer casilla en 'genres' 


      budget  ...                                               crew 
0     19995  ...  [{'credit_id': '52fe48009251416c750aca23', 'de... 
1       285  ...  [{'credit_id': '52fe4232c3a36847f800b579', 'de... 
2    206647  ...  [{'credit_id': '54805967c3a36829b5002c41', 'de... 
3     49026  ...  [{'credit_id': '52fe4781c3a36847f81398c3', 'de... 
4     49529  ...  [{'credit_id': '52fe479ac3a36847f813eaa3', 'de... 
 
[5 rows x 7 columns] 
<class 'pandas.core.frame.DataFrame'> 
Int64Index: 4808 entries, 0 to 4807 
Data columns (total 7 columns): 
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  -----  
 0   movie_id  4808 non-null   int64  
 1   title     4808 non-null   object 
 2   overview  4805 non-null   object 
 3   genres    4808 non-null   object 
 4   keywords  4808 non-null   object 
 5   cast      4808 non-null   object 
 6   crew      4808 non-null   object 
dtypes: int64(1), object(6) 
memory usage: 300.5+ KB 
None 
movie_id    0 
title       0 
overview    

In [4]:
import ast # Importamos nueva libreria 
# La siguiente función nos devuelve las palabras claves de cada cadena, es decir, pasar de esto {'id': 28, 'name': 'Action'} a esto 'Action' 
def convert(obj): # Solo aplicable a 'genres' y 'keywords' 
    L = [] 
    for i in ast.literal_eval(obj): 
        L.append(i['name']) 
    return L 
peliculas_df['genres'] = peliculas_df['genres'].apply(convert) # Nos devuelve los generos 
peliculas_df['keywords'] = peliculas_df['keywords'].apply(convert) # Nos devuelve las palabras clave 
print (peliculas_df.head()) # Revisión de información 
# La siguiente función nos devuelve las palabras claves de cada cadena, es decir, pasar de esto {'id': 28, 'name': 'Action'} a esto 'Action' 
def convert3(obj): # Nos devuelve solo los 3 primeros nombres, solo util para 'cast' 
    L = [] 
    counter = 0 
    for i in ast.literal_eval(obj): 
        if counter != 3: 
            L.append(i['name']) 
            counter +=1 
        else: 
            break 
    return L 
peliculas_df['cast'] = peliculas_df['cast'].apply(convert3) 
print (peliculas_df.head()) # Revisión de información 
def fetch_director(obj): # Nos devuelve el nombre del director 
    L = [] 
    for i in ast.literal_eval(obj): 
        if i['job'] == 'Director': 
            L.append(i['name']) 
    return L 
peliculas_df['crew'] = peliculas_df['crew'].apply(fetch_director) 
print (peliculas_df) # Revisión de información 
print (peliculas_df['overview'][0]) 


      movie_id  ...                                      crew 
0        19995  ...                           [James Cameron] 
1          285  ...                          [Gore Verbinski] 
2       206647  ...                              [Sam Mendes] 
3        49026  ...                       [Christopher Nolan] 
4        49529  ...                          [Andrew Stanton] 
       ...  ...                                       ... 
4803      9367  ...                        [Robert Rodriguez] 
4804     72766  ...                            [Edward Burns] 
4805    231617  ...                             [Scott Smith] 
4806    126186  ...                             [Daniel Hsia] 
4807     25975  ...  [Brian Herzlinger, Jon Gunn, Brett Winn] 
 
[4805 rows x 7 columns] 
In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. 
